In [2]:
import requests
from typing import Dict, Any, Optional
import geopandas as gpd
from geopandas import GeoDataFrame

BASE_URL = "http://10.32.1.107:5300/api/v1"

def get_functional_zones(
    territory_id: int,
    year: Optional[int] = None,
    source: Optional[str] = None,
    functional_zone_type_id: Optional[int] = None,
    include_child_territories: bool = False,
    cities_only: bool = False
) -> GeoDataFrame:
    """
    Получение функциональных зон для указанной территории в виде GeoDataFrame
    """
    url = f"{BASE_URL}/territory/{territory_id}/functional_zones"
    
    # Параметры запроса
    params = {
        "include_child_territories": include_child_territories,
        "cities_only": cities_only,
        "source":'OSM'
    }
    
    if year is not None:
        params["year"] = year
    if functional_zone_type_id is not None:
        params["functional_zone_type_id"] = functional_zone_type_id

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        # Предполагаем, что API возвращает список словарей с геометрией
        data = response.json()
        # Преобразуем в GeoDataFrame
        gdf = gpd.GeoDataFrame.from_features(data) if data else gpd.GeoDataFrame()
        return gdf
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе функциональных зон: {e}")
        return gpd.GeoDataFrame()
    except ValueError as e:
        print(f"Ошибка при преобразовании в GeoDataFrame: {e}")
        return gpd.GeoDataFrame()

def get_all_territories(
    parent_id: Optional[int] = None,
    get_all_levels: bool = False,
    territory_type_id: Optional[int] = None,
    name: Optional[str] = None,
    cities_only: bool = False,
    created_at: Optional[str] = None,
    centers_only: bool = False
) -> GeoDataFrame:
    """
    Получение списка всех территорий в виде GeoDataFrame
    """
    url = f"{BASE_URL}/all_territories"
    
    # Параметры запроса
    params = {
        "get_all_levels": get_all_levels,
        "cities_only": cities_only,
        "centers_only": centers_only
    }
    
    if parent_id is not None:
        params["parent_id"] = parent_id
    if territory_type_id is not None:
        params["territory_type_id"] = territory_type_id
    if name is not None:
        params["name"] = name
    if created_at is not None:
        params["created_at"] = created_at

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        # Предполагаем, что API возвращает GeoJSON
        data = response.json()
        # Преобразуем GeoJSON в GeoDataFrame
        gdf = gpd.GeoDataFrame.from_features(data["features"]) if data.get("features") else gpd.GeoDataFrame()
        return gdf
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе территорий: {e}")
        return gpd.GeoDataFrame()
    except ValueError as e:
        print(f"Ошибка при преобразовании в GeoDataFrame: {e}")
        return gpd.GeoDataFrame()

In [3]:
# Пример 1: Получение функциональных зон для территории с ID 1
zones_gdf = get_functional_zones(
    territory_id=1,
    year=2025,
    source="OSM",
    include_child_territories=True
)
print("Функциональные зоны (GeoDataFrame):")
zones_gdf.head()


Функциональные зоны (GeoDataFrame):


,geometry,landuse_zon,id
0,"POLYGON ((30.10762 58.6112, 30.10865 58.61039,...",Residential,NaN
1,"POLYGON ((30.38304 60.05619, 30.38323 60.05639...",Business,NaN
2,"POLYGON ((30.38514 60.05692, 30.38478 60.05674...",Recreation,NaN
3,"POLYGON ((30.38084 60.05327, 30.38077 60.05328...",Recreation,NaN
4,"POLYGON ((30.38565 60.05967, 30.38667 60.05938...",Recreation,NaN


In [4]:
zones_gdf = zones_gdf.set_crs(4326)

In [ ]:
zones_gdf.explore()

In [5]:
territories_gdf = get_all_territories(
    parent_id=None,
    get_all_levels=True,
    cities_only=False
)
territories_gdf.to_parquet('territories_gdf.parquet')

KeyboardInterrupt: 

In [4]:
territories_gdf

,geometry,territory_id,territory_type,parent,name,level,properties,admin_center,target_city_type,okato_code,oktmo_code,is_city,created_at,updated_at
0,"POLYGON ((34.32834 59.19564, 34.32777 59.19548...",2,"{'id': 2, 'name': 'Муниципальное образование'}","{'id': 1, 'name': 'Ленинградская область'}",Бокситогорский муниципальный район,3,"{'admin_center': None, 'Административный центр...","{'id': 328.0, 'name': 'город Бокситогорск'}",None,41203000000,41603000,False,2024-06-16T21:35:40.801621Z,2024-11-14T12:17:18.795643Z
1,"POLYGON ((34.42169 59.68391, 34.42428 59.6715,...",3,"{'id': 3, 'name': 'Поселение'}","{'id': 2, 'name': 'Бокситогорский муниципальны...",Самойловское сельское поселение,4,"{'admin_center': None, 'Административный центр...","{'id': 310.0, 'name': 'поселок Совхозный'}",None,41203876000,41603476,False,2024-06-16T21:35:40.801621Z,2024-11-14T12:17:52.703740Z
2,"POLYGON ((34.30682 59.71494, 34.30791 59.71295...",4,"{'id': 3, 'name': 'Поселение'}","{'id': 2, 'name': 'Бокситогорский муниципальны...",Большедворское сельское поселение,4,"{'admin_center': None, 'Административный центр...","{'id': 427.0, 'name': 'деревня Большой Двор'}",None,41203812000,41603412,False,2024-06-16T21:35:40.801621Z,2024-11-14T12:17:34.396287Z
3,"POLYGON ((34.14251 59.48759, 34.13616 59.49064...",5,"{'id': 3, 'name': 'Поселение'}","{'id': 2, 'name': 'Бокситогорский муниципальны...",Пикалевское городское поселение,4,"{'admin_center': None, 'Административный центр...","{'id': 335.0, 'name': 'город Пикалево'}",None,41440000000,41603102,False,2024-06-16T21:35:40.801621Z,2024-11-14T12:17:34.930089Z
4,"POLYGON ((34.05997 59.15087, 34.06007 59.14658...",6,"{'id': 3, 'name': 'Поселение'}","{'id': 2, 'name': 'Бокситогорский муниципальны...",Борское сельское поселение,4,"{'admin_center': None, 'Административный центр...","{'id': 2859.0, 'name': 'деревня Бор'}",None,41203816000,41603416,False,2024-06-16T21:35:40.801621Z,2024-11-14T12:17:35.367513Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3131,"POLYGON ((31.26917 59.17068, 31.26868 59.16717...",3133,"{'id': 8, 'name': 'Деревня'}","{'id': 205, 'name': 'Трубникоборское сельское ...",деревня Апраксин Бор,5,"{'admin_center': None, 'Административный центр...",None,None,41248844010,41648444111,True,2024-06-16T21:35:40.801621Z,2024-11-29T11:30:00.894489Z
3132,"MULTIPOLYGON (((31.32318 59.18845, 31.32312 59...",3134,"{'id': 1, 'name': 'Субъект Федерации'}","{'id': 205, 'name': 'Трубникоборское сельское ...",деревня Александровка,5,"{'admin_center': None, 'Административный центр...",None,None,41248844002,41648444106,True,2024-06-16T21:35:40.801621Z,2025-05-01T16:41:28.050780Z
3133,"POLYGON ((31.50972 59.29449, 31.5092 59.29098,...",3135,"{'id': 8, 'name': 'Деревня'}","{'id': 205, 'name': 'Трубникоборское сельское ...",деревня Большая Горка,5,"{'admin_center': None, 'Административный центр...",None,None,41248860002,41648444131,True,2024-06-16T21:35:40.801621Z,2024-11-29T11:30:01.099577Z
3134,"POLYGON ((31.49776 59.30349, 31.49724 59.29998...",3136,"{'id': 8, 'name': 'Деревня'}","{'id': 205, 'name': 'Трубникоборское сельское ...",деревня Дроздово,5,"{'admin_center': None, 'Административный центр...",None,None,41248860003,41648444146,True,2024-06-16T21:35:40.801621Z,2024-11-29T11:30:01.193353Z


In [3]:
territories_gdf = gpd.read_parquet('territories_gdf.parquet')

In [9]:
gpsp_gdf = territories_gdf[territories_gdf['level'] == 5]
gpsp_gdf


,geometry,territory_id,territory_type,parent,name,level,properties,admin_center,target_city_type,okato_code,oktmo_code,is_city,created_at,updated_at
205,"POLYGON ((33.7941 59.36206, 33.79334 59.35856,...",207,"{'id': 8, 'name': 'Деревня'}","{'id': 6, 'name': 'Борское сельское поселение'}",деревня Болото,5,"{'admin_center': None, 'Административный центр...",None,None,41203816004,41603416106,True,2024-06-16T21:35:40.801621Z,2024-11-29T11:30:08.351607Z
206,"POLYGON ((33.82129 59.47496, 33.82053 59.47146...",208,"{'id': 8, 'name': 'Деревня'}","{'id': 6, 'name': 'Борское сельское поселение'}",деревня Большой Остров,5,"{'admin_center': None, 'Административный центр...",None,None,41203816020,41603416111,True,2024-06-16T21:35:40.801621Z,2024-11-29T11:30:01.400490Z
207,"MULTIPOLYGON (((33.79297 59.47584, 33.79336 59...",209,"{'id': 1, 'name': 'Субъект Федерации'}","{'id': 6, 'name': 'Борское сельское поселение'}",деревня Бор,5,"{'admin_center': None, 'Административный центр...",None,None,41203816001,41603416101,True,2024-06-16T21:35:40.801621Z,2025-05-01T16:33:16.434840Z
208,"POLYGON ((33.81098 59.44228, 33.81022 59.43878...",210,"{'id': 8, 'name': 'Деревня'}","{'id': 6, 'name': 'Борское сельское поселение'}",деревня Бороватое,5,"{'admin_center': None, 'Административный центр...",None,None,41203816005,41603416116,True,2024-06-16T21:35:40.801621Z,2024-11-29T11:30:02.071574Z
209,"MULTIPOLYGON (((33.67043 59.32827, 33.66769 59...",211,"{'id': 1, 'name': 'Субъект Федерации'}","{'id': 6, 'name': 'Борское сельское поселение'}",деревня Бочево,5,"{'admin_center': None, 'Административный центр...",None,None,41203816003,41603416121,True,2024-06-16T21:35:40.801621Z,2025-05-01T15:32:08.695376Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3131,"POLYGON ((31.26917 59.17068, 31.26868 59.16717...",3133,"{'id': 8, 'name': 'Деревня'}","{'id': 205, 'name': 'Трубникоборское сельское ...",деревня Апраксин Бор,5,"{'admin_center': None, 'Административный центр...",None,None,41248844010,41648444111,True,2024-06-16T21:35:40.801621Z,2024-11-29T11:30:00.894489Z
3132,"MULTIPOLYGON (((31.32318 59.18845, 31.32312 59...",3134,"{'id': 1, 'name': 'Субъект Федерации'}","{'id': 205, 'name': 'Трубникоборское сельское ...",деревня Александровка,5,"{'admin_center': None, 'Административный центр...",None,None,41248844002,41648444106,True,2024-06-16T21:35:40.801621Z,2025-05-01T16:41:28.050780Z
3133,"POLYGON ((31.50972 59.29449, 31.5092 59.29098,...",3135,"{'id': 8, 'name': 'Деревня'}","{'id': 205, 'name': 'Трубникоборское сельское ...",деревня Большая Горка,5,"{'admin_center': None, 'Административный центр...",None,None,41248860002,41648444131,True,2024-06-16T21:35:40.801621Z,2024-11-29T11:30:01.099577Z
3134,"POLYGON ((31.49776 59.30349, 31.49724 59.29998...",3136,"{'id': 8, 'name': 'Деревня'}","{'id': 205, 'name': 'Трубникоборское сельское ...",деревня Дроздово,5,"{'admin_center': None, 'Административный центр...",None,None,41248860003,41648444146,True,2024-06-16T21:35:40.801621Z,2024-11-29T11:30:01.193353Z


In [8]:
import requests
from typing import Optional
import geopandas as gpd
from geopandas import GeoDataFrame

BASE_URL = "http://10.32.1.107:5300/api/v1"

def get_physical_objects_geojson(
    territory_id: int,
) -> GeoDataFrame:

    url = f"http://10.32.1.107:5300/api/v1/territory/{territory_id}/physical_objects_geojson?include_child_territories=false&cities_only=false&centers_only=false&physical_object_function_id=1"

    try:
        response = requests.get(url)
        response.raise_for_status()
        
        # Предполагаем, что API возвращает GeoJSON с полем "features"
        data = response.json()
        features = data.get("features", [])
        
        if not features:
            return gpd.GeoDataFrame()
            
        gdf = gpd.GeoDataFrame.from_features(features)
        gdf['territory_id'] = territory_id
        return gdf
    except:
        print(f"Ошибка при запросе")
        return gpd.GeoDataFrame()


In [ ]:
from tqdm import tqdm

gpsp_ids = gpsp_gdf['territory_id'].unique()
physical_objects_gdf_list = []
for id in tqdm(gpsp_ids):
    physical_objects_gdf_list.append(get_physical_objects_geojson(territory_id=id))
buildings_gdf = gpd.pd.concat(physical_objects_gdf_list, axis=0, ignore_index=True)
buildings_gdf.to_parquet('buildings_gdf.parquet')

  0%|          | 0/188 [00:00<?, ?it/s]

100%|██████████| 188/188 [04:05<00:00,  1.31s/it]


In [57]:
buildings_gdf = gpd.read_parquet('buildings_gdf.parquet')

In [44]:
buildings_gdf = buildings_gdf[buildings_gdf['building'].notna()]

In [58]:
buildings_gdf = buildings_gdf.join(gpd.pd.json_normalize(buildings_gdf['building']))
buildings_gdf = buildings_gdf.drop(columns=['building']) 

living_building_type_id = 4
key_to_check = 'physical_object_type_id'  
buildings_gdf['is_living'] = buildings_gdf.apply(lambda row: row['physical_object_type'].get('physical_object_type_id') == living_building_type_id, axis=1)

In [65]:
columns = ['geometry','territory_id','is_living','building_area_official',
           'properties.storeys_count','floors',
           'properties.population_balanced',]

buildings_gdf = buildings_gdf[columns]

In [66]:
buildings_gdf['is_living'].sum()

np.int64(17583)

In [71]:
# Разделяем DataFrame на два по значению is_living
true_df = buildings_gdf[buildings_gdf['is_living'] == True]
false_df = buildings_gdf[buildings_gdf['is_living'] == False]

# Считаем количество не-NaN значений для каждой группы
columns_to_count = [
    'building_area_official',
    'properties.storeys_count',
    'floors',
    'properties.population_balanced'
]

true_counts = true_df[columns_to_count].count()
false_counts = false_df[columns_to_count].count()

# Выводим результат
print("Количество ненулевых значений для is_living == True:")
print(true_counts)
print("\nКоличество ненулевых значений для is_living == False:")
print(false_counts)

# Если нужно общее количество строк в каждой группе
print(f"\nВсего строк с is_living == True: {len(true_df)}")
print(f"Всего строк с is_living == False: {len(false_df)}")

Количество ненулевых значений для is_living == True:
building_area_official            10613
properties.storeys_count            790
floors                              918
properties.population_balanced    10613
dtype: int64

Количество ненулевых значений для is_living == False:
building_area_official            0
properties.storeys_count          0
floors                            0
properties.population_balanced    0
dtype: int64

Всего строк с is_living == True: 17583
Всего строк с is_living == False: 134338


In [26]:
import geopandas as gpd
import pandas as pd

# Отделяем агрегацию числовых колонок
numeric_agg_dict = {
    'building_area_modeled': ['sum', 'mean'],
    'properties.storeys_count': ['sum', 'mean'],
    'properties.population_balanced': ['sum', 'mean'],
    'properties.living_area_modeled': ['sum', 'mean'],
    'floors': ['sum', 'mean']
}

# Агрегируем числовые колонки
numeric_agg = buildings_gdf.groupby('territory_id').agg(numeric_agg_dict)

# Исправляем названия колонок для числовых данных
numeric_agg.columns = ['_'.join(col).strip() for col in numeric_agg.columns.values]

# Получаем первые значения для нечисловых колонок
first_values = buildings_gdf.groupby('territory_id').agg({
    'geometry': 'first',
    'is_living': 'first'
}).reset_index()

# Объединяем результаты
aggregated_gdf = first_values.merge(numeric_agg.reset_index(), on='territory_id')

# Преобразуем обратно в GeoDataFrame
aggregated_gdf = gpd.GeoDataFrame(aggregated_gdf, geometry='geometry')

# Если нужно сохранить CRS из исходного gdf
if buildings_gdf.crs:
    aggregated_gdf.crs = buildings_gdf.crs